In [ ]:
import re
import numpy as np


with open("6/input.txt", "r") as inpt:
    inpt = inpt.readlines()
    scroll = np.array([[int(num) for num in re.findall(r"\d+", line)]
                       for line in inpt[:-1]])
    operations = np.array([op for op in re.findall(r"[+*]", inpt[-1])])

scroll = scroll.transpose()

solution = []
for i, line in enumerate(scroll):
    if operations[i] == "+":
        solution.append(np.sum(line))
    elif operations[i] == "*":
        solution.append(np.prod(line))

print(solution)

[33210, 490, 4243455, 401]


5